In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass, field
from typing import Dict, Optional

In [ ]:
def config_domain(NX, NY, W, H, AXI=0):
    global NI, NJ, IDIM, JDIM, IJDIM, NIM, NJM, DX0, DY0, IR
    IR = AXI
    
    NI = NX
    NJ = NY
    IDIM = NI + 10
    JDIM = NJ + 10
    IJDIM = max(IDIM, JDIM)
    NIM   = NI - 1
    NJM   = NJ - 1
    
    DX0 = W / NI
    DY0 = H / NJ
    
def init_domain():
   global IDIM, JDIM, IJDIM
   global NI, NIM, NJ, NJM
   global XP, XU, YP, YV
   global DX0, DY0
   global DXP, DXU, DYP, DYV
   global RP, RU

   # DOMAIN ARRAYS
   XP  = np.zeros(IDIM)
   XU  = np.zeros(IDIM)
   
   ## CONTROL VOLUME SIZE ARRAYS
   DXP = np.zeros(IDIM) 
   DXU = np.zeros(IDIM)
   
   YP  = np.zeros(JDIM)
   YV  = np.zeros(JDIM)
   DYP = np.zeros(JDIM)
   DYV = np.zeros(JDIM)
   
   RP  = np.zeros(IDIM)
   RU  = np.zeros(JDIM)

   ## initialize domain sizes
   for i in range(1,NI+1):  XU[i]=DX0*(i-1)
   for i in range(1,NI):    XP[i]=(XU[i]+XU[i+1])/2
   for j in range(1,NJ+1):  YV[j]=DY0*(j-1)
   for j in range(1,NJ):    YP[j]=(YV[j]+YV[j+1])/2

   XP[0]=XU[1]
   XP[NI]=XU[NI]
   YP[0]=YV[1]
   YP[NJ]=YV[NJ]

   ## initialize control volume sizes
   for i in range(1,NI):    DXP[i]=XU[i+1]-XU[i]
   for i in range(1,NI+1):  DXU[i]=XP[i]-XP[i-1]
   for j in range(1,NJ):    DYP[j]=YV[j+1]-YV[j]
   for j in range(1,NJ+1):  DYV[j]=YP[j]-YP[j-1]

   ## initialize face dimensions
   if IR==0:
      for i in range(1,NI+1): RU[i]=1
      for i in range(0,NI+1): RP[i]=1
   else:
      for i in range(1,NI+1): RU[i]=np.fabs(XU[i])
      for i in range(0,NI+1): RP[i]=np.fabs(XP[i])

def set_properties(RHO0, MU0, CP0, K0, QDOT0):
    global RHO, MU, CP, K, QDOT
    RHO   = RHO0
    MU    = MU0
    CP    = CP0
    K     = K0
    QDOT  = QDOT0

@dataclass
class BoundaryValues:
    N: Optional[float] = None
    S: Optional[float] = None
    E: Optional[float] = None
    W: Optional[float] = None

@dataclass
class FieldEquation:
    name: str
    # Relaxation Factor
    relax: float
    # Quantity
    PHI: np.ndarray = None
    # Coefficients
    AP: np.ndarray = field(default=None)
    AW: np.ndarray = field(default=None)
    AE: np.ndarray = field(default=None)
    AS: np.ndarray = field(default=None)
    AN: np.ndarray = field(default=None)
    # Boundary Cond
    BC: BoundaryValues = field(default_factory=BoundaryValues)
    # Neumann Option
    neumann: Optional[Dict[str, Optional[str]]] = None
    def __getitem__(self, idx):
        return self.PHI[idx]     # e.g., phi[i], phi[i,j]

    def __setitem__(self, idx, value):
        self.PHI[idx] = value   # e.g., phi[i,j] = value
        
    def __post_init__(self):
        self.PHI = np.zeros((IDIM, JDIM))
        self.AP = np.zeros((IDIM, JDIM))
        self.AW = np.zeros((IDIM, JDIM))
        self.AE = np.zeros((IDIM, JDIM))
        self.AS = np.zeros((IDIM, JDIM))
        self.AN = np.zeros((IDIM, JDIM))

In [ ]:
def divideByZeroWarn(val,funcName):
    if np.abs(val)<=1e-30:
        message="WARNING: denominator {} is zero!!".format(val)
        message+="\t FROM:{}".format(funcName)
        print(message)

def TDMA(start, end, AW, AE, AP, RHS_X):
    X = np.zeros(len(AP))
    c_prime = np.zeros(IJDIM)
    d_prime = np.zeros(IJDIM)
    
    denom=AP[start]
    divideByZeroWarn(denom,TDMA.__name__)

    c_prime[start] = AE[start] / denom
    d_prime[start] = RHS_X[start] / denom
    
    for i in range(start + 1, end + 1):

        denom=(AP[i] - AW[i] * c_prime[i - 1])
        divideByZeroWarn(denom,TDMA.__name__)

        c_prime[i] = AE[i] / denom
        d_prime[i] = (RHS_X[i] - AW[i] * d_prime[i - 1]) / denom
    X[end] = d_prime[end]
    for i in range(end - 1, start - 1, -1):
        X[i] = d_prime[i] - c_prime[i] * X[i + 1]

    return X

def LLTDMA(i_start, i_end, j_start, j_end, AW, AE, AS, AN, AP, RHS):
    RHS_x = np.zeros(IDIM)
    RHS_y = np.zeros(JDIM)
    X = np.zeros((IDIM, JDIM))

    for iter in range(2):
        # TDMA: x-direction
        for j in range(j_start, j_end+1):
            for i in range(i_start, i_end + 1):
                neighbors = AS[i][j] * X[i][j - 1] + AN[i][j] * X[i][j + 1]
                RHS_x[i] = RHS[i][j] + neighbors
            X[:, j]=TDMA(i_start, i_end, -AW[:, j], -AE[:, j], AP[:, j], RHS_x)
    
        # TDMA: y-direction
        for i in range(i_start, i_end+1):
            for j in range(j_start, j_end + 1):
                neighbors = AW[i][j] * X[i - 1][j] + AE[i][j] * X[i + 1][j]
                RHS_y[j] = RHS[i][j] + neighbors
            X[i, :] = TDMA(j_start, j_end, -AS[i, :], -AN[i, :], AP[i, :], RHS_y)

    return X

In [ ]:
def set_BC(PHI):
    for j in range(0, NJ): PHI[0][j]  =  PHI.BC.W  # West Wall
    for j in range(0, NJ): PHI[NI][j] =  PHI.BC.E  # East Wall
    for i in range(0, NI): PHI[i][0]  =  PHI.BC.S  # South Wall
    for i in range(0, NI): PHI[i][NJ] =  PHI.BC.N  # North Wall

In [ ]:
def set_NEUMANN(PHI, flag):
    if PHI.neumann is None:
        return
    
    if flag == "pre":
        for j in range(1, NJ):
            if PHI.neumann.get("W") == "neumann": 
                PHI.AP[1][j] -= PHI.AW[1][j]
                PHI.AW[1][j] = 0
            if PHI.neumann.get("E") == "neumann": 
                PHI.AP[NIM][j] -= PHI.AE[NIM][j]
                PHI.AE[NIM][j] = 0
        for i in range(1, NI):
            if PHI.neumann.get("S") == "neumann": 
                PHI.AP[i][1] -= PHI.AS[i][1]
                PHI.AS[i][1] = 0
            if PHI.neumann.get("N") == "neumann": 
                PHI.AP[i][NJM] -= PHI.AN[i][NJM]
                PHI.AN[i][NJM] = 0 
        return

    if flag == "post":
        for j in range(1, NJ):
            if PHI.neumann.get("W") == "neumann": PHI[0][j]  = PHI[1][j]
            if PHI.neumann.get("E") == "neumann": PHI[NI][j] = PHI[NIM][j]
        for i in range(1, NI):
            if PHI.neumann.get("S") == "neumann": PHI[i][0]  = PHI[i][1]
            if PHI.neumann.get("N") == "neumann": PHI[i][NJ] = PHI[i][NJM]
        return

### upwind velocity

\begin{equation*}
U^*_{i+\frac{1}{2}} = 
\left\{ 
\begin{array}{rcl}
U_i & U_{i+\frac{1}{2}} > 0 \\
U_{i+1} & U_{i+\frac{1}{2}} < 0
\end{array}
\right\}
\end{equation*}


In [ ]:
def outOfRangeWarn(i,j):
    if i>NI:
        print("WARNING: index i={0} Out Of Range NI={1}".format(i,NI))
    if j>NJ:
        print("WARNING: index j={0} Out Of Range NJ={1}".format(j,NJ))

def minmod(r):
    return np.fmax(0,np.fmin(1,r))

def get_faceVal(PHI, face_vel, i, j, dir, scheme=None):  # MUSCL ALGORITHM
    if dir not in ["x","y"]: AssertionError
    if face_vel<0: 
        if dir=="x":u_i, u_j = (i+1,j)
        elif dir=="y":u_i, u_j = (i,j+1)
        sign=-1
    else:
        u_i, u_j = (i,j)
        sign=1

    phi_cur=PHI[u_i][u_j]
    if dir=="x": 
        denom=PHI[u_i][u_j]-PHI[u_i-1][u_j]+1e-30  
        numer=PHI[u_i+1][u_j]-PHI[u_i][u_j]
    else:        
        denom=PHI[u_i][u_j]-PHI[u_i][u_j-1]+1e-30
        numer=PHI[u_i][u_j+1]-PHI[u_i][u_j]
    # divideByZeroWarn(denom,get_faceVal.__name__)
    r=numer/denom

    return phi_cur+0.5*sign*denom*minmod(r)

def get_RES(PHI):
    global U, V
    global RHS, RES
    
    # b: Flux Coeff
    if PHI.name=="T":           b=RHO * CP
    elif PHI.name in ["U","V"]: b=RHO
    
    for i in range(1, NI):
        for j in range(1, NJ):
            
            ## advection terms
            # upwind criteria (faced flux)
            if PHI.name=="T":  
                uw=U[i][j]
                ue=U[i+1][j]
                vs=V[i][j]
                vn=V[i+1][j]
            elif PHI.name =="U":
                uw=0.5*(U[i][j]+U[i-1][j])
                ue=0.5*(U[i][j]+U[i+1][j])
                vs=0.5*(V[i-1][j]+V[i][j])
                vn=0.5*(V[i-1][j+1]+V[i][j+1])
            elif PHI.name =="V":
                uw=0.5*(U[i][j-1]+U[i][j])
                ue=0.5*(U[i+1][j-1]+U[i+1][j])
                vs=0.5*(V[i][j]+V[i][j-1])
                vn=0.5*(V[i][j]+V[i][j+1])

            # b: advection coeff
            if PHI.name=="T":           b=RHO * CP
            elif PHI.name in ["U","V"]: b=RHO
            # carrier flux (important)
            Fw = b * RU[i]       * DYP[j] * uw
            Fe = b * RU[i + 1]   * DYP[j] * ue
            Fs = b * RP[i]       * DXP[i] * vs
            Fn = b * RP[i]       * DXP[i] * vn

            neighbors = (
                  PHI.AW[i][j] * PHI[i - 1][j]
                + PHI.AE[i][j] * PHI[i + 1][j]
                + PHI.AS[i][j] * PHI[i][j - 1]
                + PHI.AN[i][j] * PHI[i][j + 1]
            )
            FOU = PHI.AP[i][j] * PHI[i][j] - neighbors

            # high-resolution scheme (MUSCL)
            ## upwind face value
            upwind_w = get_faceVal(PHI, uw, i-1, j, "x")
            upwind_e = get_faceVal(PHI, ue, i  , j, "x")
            upwind_s = get_faceVal(PHI, vs, i, j-1, "y")
            upwind_n = get_faceVal(PHI, vn, i, j  , "y")

            
            x_advect_HO = Fe * upwind_e - Fw * upwind_w
            y_advect_HO = Fn * upwind_n - Fs * upwind_s
            x_advect_LO = (np.fmax(0, Fe) * PHI[i][j]   + np.fmax(0, -Fe) * PHI[i+1][j]) \
                       - (np.fmax(0, Fw) * PHI[i-1][j] + np.fmax(0, -Fw) * PHI[i][j])
            y_advect_LO = (np.fmax(0, Fn) * PHI[i][j]   + np.fmax(0, -Fn) * PHI[i][j+1]) \
                       - (np.fmax(0, Fs) * PHI[i][j-1] + np.fmax(0, -Fs) * PHI[i][j])
            x_advect = x_advect_HO - x_advect_LO
            y_advect = y_advect_HO - y_advect_LO
            advect = x_advect + y_advect

            SOU = FOU + advect

            # residual & correction
            RES[i][j] = RHS[i][j] - SOU

    return RES

def SOLVE(PHI):
    global U, V, P
    global RHS
    global NI, NJ, NIM, NJM
    global QDOT, RHO, CP, K0, MU0
    
    if PHI.name not in ["U","V","P","T"]: AssertionError

    # TDMA coeffs
    for i in range(1, NI):
        for j in range(1, NJ):
            
            # diffusion terms
            ## a: diffusion Coeff
            if PHI.name=="T":           a=K
            elif PHI.name in ["U","V"]: a=MU
            ## LHS_diffusion
            AW_diff = a * RU[i] * DYP[j] / DXU[i]
            AE_diff = a * RU[i + 1] * DYP[j] / DXU[i + 1]
            AS_diff = a * RP[i] * DXP[i] / DYV[j]
            AN_diff = a * RP[i] * DXP[i] / DYV[j + 1]

            # advection terms
            ## upwind criteria (faced flux)
            if PHI.name=="T":  
                uw=U[i][j]
                ue=U[i+1][j]
                vs=V[i][j]
                vn=V[i+1][j]
            elif PHI.name =="U":
                uw=0.5*(U[i][j]+U[i-1][j])
                ue=0.5*(U[i][j]+U[i+1][j])
                vs=0.5*(V[i-1][j]+V[i][j])
                vn=0.5*(V[i-1][j+1]+V[i][j+1])
            elif PHI.name =="V":
                uw=0.5*(U[i][j-1]+U[i][j])
                ue=0.5*(U[i+1][j-1]+U[i+1][j])
                vs=0.5*(V[i][j]+V[i][j-1])
                vn=0.5*(V[i][j]+V[i][j+1])

            ## b: advection coeff
            if PHI.name=="T":           b=RHO * CP
            elif PHI.name in ["U","V"]: b=RHO
            ## carrier flux (important)
            Fw = b * RU[i]       * DYP[j] * uw
            Fe = b * RU[i + 1]   * DYP[j] * ue
            Fs = b * RP[i]       * DXP[i] * vs
            Fn = b * RP[i]       * DXP[i] * vn

            # LHS = LHS_diffusion + LHS_advection
            PHI.AW[i][j] = AW_diff + np.fmax(0, +Fw)
            PHI.AE[i][j] = AE_diff + np.fmax(0, -Fe)
            PHI.AS[i][j] = AS_diff + np.fmax(0, +Fs)
            PHI.AN[i][j] = AN_diff + np.fmax(0, -Fn)
            PHI.AP[i][j] = PHI.AW[i][j] + PHI.AE[i][j] + PHI.AS[i][j] + PHI.AN[i][j]
            
            # RHS: regarded as "answer", latest updated
            if PHI.name == "T": 
                RHS[i][j] = QDOT * RP[i] * DXP[i] * DYP[j]
            ## core advantage of STAGGERED-GRID
            elif PHI.name == "U":
                RHS[i][j] = -(P[i][j]-P[i-1][j])*DYP[j]*DXU[i]
            elif PHI.name == "V":
                RHS[i][j] = -(P[i][j]-P[i][j-1])*DXP[i]*DYV[j]
            
    # boundary conds
    set_NEUMANN(PHI, "pre")
    
    RES = get_RES(PHI)
    CORR = LLTDMA(1, NIM, 1, NJM, PHI.AW, PHI.AE, PHI.AS, PHI.AN, PHI.AP, RHS=RES)
    
    sl = (slice(1, NI), slice(1, NJ))
    FFMAX = max(1.0e-30,np.max(np.abs(PHI[sl])))
    RESMAX = np.max(np.abs(RES[sl] / PHI.AP[sl]))/FFMAX
    FCMAX = np.max(np.abs(CORR[sl]))/FFMAX
    
    for i in range(1, NI):
        for j in range(1, NJ):
            PHI[i][j] += PHI.relax * CORR[i][j]
            
    return (RESMAX, FCMAX)
   

### TODO List

1. Upwind criteria for U,V --> flux-based selection
2. Staggered Grid description --> U,V control volume
3. Non-uniform Grid
4. Theory 1, Theory 2
5. Difference of Carrier vs Cargo velocity
   * linked with SIMPLE
     * Carrier: u^n
     * Cargo: u* 

\begin{equation*}
\begin{align*}
LHS_{\text{advection}}
&=(F_e^+ (u_{i,j}) + F_e^- (u_{i+1,j})) \\
&-(F_w^+ (u_{i-1,j}) + F_w^- (u_{i,j})) \\
&+(F_n^+ (u_{i,j}) + F_n^- (u_{i,j+1})) \\
&-(F_s^+ (u_{i,j-1}) + F_s^- (u_{i,j})) \\
\end{align*}
\end{equation*}

Remind that $AP=AW+AE+AS+AN$ so that we finally set LHS as,

\begin{equation*}
LHS_{\text{advection}}=AP\cdot U_{i,j}-(AW\cdot U_{i-1,j}+AE\cdot U_{i+1,j}+AS\cdot U_{i,j-1}+AN\cdot U_{i,j+1})
\end{equation*}

Then,

\begin{equation*}
\begin{align*}
AW&=+F_w^- \\
AE&=-F_e^+ \\
AS&=+F_s^- \\
AN&=-F_n^+ \\
AP&=AW+AE+AS+AN
\end{align*}
\end{equation*}


\begin{equation*}
\begin{align*}
LHS&=LHS_{\text{advection}}+LHS_{\text{diffusion}} \\
&LHS_{\text{advection}} \qquad \text{(second order implicit)} \\
&LHS_{\text{diffusion}} \qquad \text{(first order explicit)} \\
\end{align*}
\end{equation*}

Diffusion terms (also) earned faces, [i+1,i], [i,i-1], [j+1,j], [j,j-1]

#### **x-directional velocity U**

\begin{equation*}
\begin{align*}
 & \quad \rho \left( u^n \frac{\partial u^*}{\partial x} + v^n \frac{\partial u^*}{\partial y} \right) \\
&= \rho \left( \frac{\partial u^nu^*}{\partial x} +  \frac{\partial u^*v^n}{\partial y} \right) \qquad \text{conservative form}
\end{align*}
\end{equation*}

Since advection term $\rho(u^*u^n)$, $\rho(v^*u^n)$ can be regarded as result of product: Carrier & Cargo:

\begin{equation*}
\begin{align*}
\underbrace{(\rho u^n)}_\text{Carrier}\cdot \underbrace{u^*}_\text{Cargo} \qquad
\underbrace{(\rho v^n)}_\text{Carrier}\cdot \underbrace{u^*}_\text{Cargo}
\end{align*}
\end{equation*}

Then discrete form:

\begin{equation*}
\begin{align*}
\left[ (\rho u^n_{i+\frac{1}{2},j})u^*_{i+\frac{1}{2},j} - (\rho u^n_{i-\frac{1}{2},j})u^*_{i-\frac{1}{2},j} \right] \Delta y +
\left[ (\rho v^n_{i,j+\frac{1}{2}})u^*_{i,j+\frac{1}{2}} - (\rho v^n_{i,j-\frac{1}{2}})u^*_{i,j-\frac{1}{2}} \right] \Delta x
\end{align*}
\end{equation*}

Simply set $\rho u \Delta x$ as Flux:

\begin{equation*}
\begin{align*}
\left[ F_e u^*_{i+\frac{1}{2},j} - F_w u^*_{i-\frac{1}{2},j} \right] +
\left[ F_n u^*_{i,j+\frac{1}{2}} - F_s u^*_{i,j-\frac{1}{2}} \right] 
\end{align*}
\end{equation*}



#### **Projection Method**

Momentum Equation:

\begin{equation*}
\begin{align}
\rho\left[ u^n \frac{\partial u^{n+1}}{\partial x} + v^n \frac{\partial u^{n+1}}{\partial y} \right]
&= -\frac{\partial P^{n+1}}{\partial x} + 
\frac{\partial}{\partial x}\left( \mu \frac{\partial u^{n+1}}{\partial x} \right) + 
\frac{\partial}{\partial y}\left( \mu \frac{\partial u^{n+1}}{\partial y} \right) \quad \\

\rho\left[ u^n \frac{\partial u^{*}}{\partial x} + v^n \frac{\partial u^{*}}{\partial y} \right]
&= -\frac{\partial P^{n}}{\partial x} + \frac{\partial}{\partial x}\left( \mu \frac{\partial u^{*}}{\partial x} \right) + \frac{\partial}{\partial y}\left( \mu \frac{\partial u^{*}}{\partial y} \right) \quad
\end{align}
\end{equation*}

\begin{equation*}
\begin{align*}
(1)-(2)
&=
\rho\left[ u^n \left(\frac{\partial u^{n+1}}{\partial x}-\frac{\partial u^{*}}{\partial x}\right) + v^n \left(\frac{\partial u^{n+1}}{\partial y}-\frac{\partial u^{*}}{\partial y}\right) \right] \\
&= -\left(\frac{\partial P^{n+1}}{\partial x}-\frac{\partial P^{n}}{\partial x}\right)  
+\left(
\frac{\partial}{\partial x}\left( \mu \frac{\partial u^{n+1}}{\partial x} \right) -
\frac{\partial}{\partial x}\left( \mu \frac{\partial u^{*}}{\partial x} \right) 
\right)
+\left(
\frac{\partial}{\partial y}\left( \mu \frac{\partial u^{n+1}}{\partial y} \right) -
\frac{\partial}{\partial y}\left( \mu \frac{\partial u^{*}}{\partial y} \right)
\right)
\end{align*}
\end{equation*}


In [ ]:
def PROJECT(flag):
    global U, V, P
    global AP_U, AP_V
    
    if flag=="pre":
        for i in range(1, NI):
            for j in range(1, NJ):
                U[i][j] += 0.5*AP_U[i][j]*(P[i][j]-P[i-1][j])
                V[i][j] += 0.5*AP_V[i][j]*(P[i][j]-P[i][j-1])
                
    elif flag=="post":
        for i in range(1, NI):
            for j in range(1, NJ):
                U[i][j] -= 0.5*AP_U[i][j]*(P[i][j]-P[i-1][j])
                V[i][j] -= 0.5*AP_V[i][j]*(P[i][j]-P[i][j-1])
    
def SOLVE_P(method="SIMPLE"):
    global U, V, P
    global AP_U, AP_V
    
    AP_U = np.zeros((IDIM,JDIM))
    AP_V = np.zeros((IDIM,JDIM))   
    RES = np.zeros((IDIM,JDIM))
    
    if method not in ["SIMPLE","SIMPLEC"]: AssertionError
    
    # remove old pressure correction
    for i in range(1, NI):
        for j in range(1, NJ):
            if method=="SIMPLE": 
                AP_U[i][j] = DYP[j]* DXU[i] / U.AP[i][j]
                AP_V[i][j] = DXP[i]* DYV[j] / V.AP[i][j]
                
            elif method=="SIMPLEC": 
                neibors_u = U.AE[i][j] + U.AW[i][j] + U.AN[i][j] + U.AS[i][j]
                neibors_v = V.AE[i][j] + V.AW[i][j] + V.AN[i][j] + V.AS[i][j]
                AP_U[i][j] = DYP[j]* DXU[i] / (U.AP[i][j]-neibors_u)
                AP_V[i][j] = DXP[i]* DYV[j] / (V.AP[i][j]-neibors_v)
                
    PROJECT(flag="pre")
    set_NEUMANN(P, "pre")
    
    for i in range(1, NI):
        for j in range(1, NJ):
            P.AW[i][j] = AP_U[i][j] * RU[i] * DYP[j] / DXU[i];
            P.AE[i][j] = AP_U[i+1][j] * RU[i + 1] * DYP[j] / DXU[i + 1];
            P.AS[i][j] = AP_V[i][j] * RP[i] * DXP[i] / DYV[j];
            P.AN[i][j] = AP_V[i][j+1] * RP[i] * DXP[i] / DYV[j + 1];
            P.AP[i][j] = P.AW[i][j]+P.AE[i][j]+P.AS[i][j]+P.AN[i][j];
            
            
    for i in range(1, NI):
        for j in range(1, NJ):
            Fw = RU[i] *     DYP[j] * U[i][j]
            Fe = RU[i + 1] * DYP[j] * U[i + 1][j]
            Fs = RP[i] *     DXP[i] * V[i][j]
            Fn = RP[i] *     DXP[i] * V[i][j + 1]
            
            RHS=Fw-Fe+Fs-Fn
            neighbors = (
                    P.AW[i][j] * P[i - 1][j]
                    + P.AE[i][j] * P[i + 1][j]
                    + P.AS[i][j] * P[i][j - 1]
                    + P.AN[i][j] * P[i][j + 1]
            )
            
            RES[i][j] = RHS - (P.AP[i][j] * P[i][j] - neighbors)
      
    CORR = LLTDMA(1, NIM, 1, NJM, P.AW, P.AE, P.AS, P.AN, P.AP, RHS=RES)
    
    sl = (slice(1, NI), slice(1, NJ))
    FFMAX = max(1.0e-30,np.max(np.abs(P[sl])))
    RESMAX = np.max(np.abs(RES[sl] / P.AP[sl]))/FFMAX
    FCMAX = np.max(np.abs(CORR[sl]))/FFMAX
    
    for i in range(1, NI):
        for j in range(1, NJ):
            P[i][j] += P.relax * CORR[i][j]
            
    set_NEUMANN(P, "post")
    PROJECT(flag="post")
    
    return (RESMAX, FCMAX)

In [ ]:
def config_solver(jump=100, errmax=1e-6):
    global JUMP, ERRMAX
    JUMP = jump
    ERRMAX = errmax

def monitor_RES(iter, resmax, fcmax):
    global U, V, P, T
    global JUMP, ERRMAX
    is_continue = 1

    ref_id = (int(NI/2), int(NJ/2))

    if iter % JUMP == 0:
        print(
            " iter= {:4d} U= {:11.3e} V= {:11.3e} P= {:11.3e} ResMax= {:11.3e} Fcmax= {:11.3e}".format(
                iter, U[ref_id], V[ref_id], P[ref_id], resmax, fcmax
            )
        )
    if iter and resmax < ERRMAX and fcmax < ERRMAX:
        print(
            " iter= {:4d} U= {:11.3e} V= {:11.3e} P= {:11.3e} ResMax= {:11.3e} Fcmax= {:11.3e}".format(
                iter, U[ref_id], V[ref_id], P[ref_id], resmax, fcmax
            )
        )
        print(" CONVERGENCE ACHIEVED!! ")
        is_continue = 0
    return is_continue

def SOLVE_ALL(max_iter=1000):
    global U, V, P, T
    iter = 0
    is_continue = 1

    for iter in range(max_iter):
        RESMAX_U, FCMAX_U = SOLVE(U)
        RESMAX_V, FCMAX_V = SOLVE(V)
        RESMAX_P, FCMAX_P = SOLVE_P(method="SIMPLE")
        
        RESMAX= max(RESMAX_U, RESMAX_V, RESMAX_P)
        FCMAX= max(FCMAX_U, FCMAX_V, FCMAX_P)
        
        if monitor_RES(iter, RESMAX, FCMAX)==0: break

In [ ]:
config_domain(NX=21, NY=81, W=5e-3, H=2.5e-2)
init_domain()
RHS=np.zeros((IDIM,JDIM))
RES=np.zeros((IDIM,JDIM))
config_solver(jump=100, errmax=1e-5)
set_properties(RHO0=1000, MU0=0.001, CP0=1000, K0=1, QDOT0=0)

U = FieldEquation(
    name = "U",
    BC  = BoundaryValues(W=0, E=0, S=0, N=0),
    neumann = {"S": "neumann", "N": "neumann"},
    relax = 1,
)

V = FieldEquation(
    name = "V",
    BC  = BoundaryValues(W=0, E=0, S=0.1, N=0),
    neumann = {"W": "nuemann", "N": "nuemann"},
    relax = 1,
)

P = FieldEquation(
    name = "P",
    BC  = BoundaryValues(W=0, E=0, S=0, N=0),
    neumann = {"W": "neumann", "E": "neumann", "S": "neumann"},
    relax = 1,
)

set_BC(U)
set_BC(V)
set_BC(P)

In [ ]:
SOLVE_ALL(max_iter=2000)